<a href="https://colab.research.google.com/github/sannihith08/Inventory_Forecast/blob/main/Combined_Strargies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Default title text
import yfinance as yf
import pandas as pd
from datetime import datetime
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import talib
!pip install ta
import ta
#!pip install --upgrade yfinance
#!pip install mplfinance
import os

# Example: delete 'myfile.csv'

# print(os.listdir('/content'))
# !rm -rf /content/*
# print("All saved files deleted successfully!")

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=955885c4bcf17c0edefd4c59c766bc8089f26fbb19b24adfc8b66054aa0099a8
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [22]:
# @title Swing Trading-Day
#Swing Trading

import yfinance as yf
import pandas as pd
import numpy as np
from ta.trend import ADXIndicator
from ta.volatility import BollingerBands
from ta.volatility import AverageTrueRange

# ✅ Symbol and Interval
nifty = "SUNPHARMA.NS"     # Example stock   --^NSEI
#nifty_S =  ["TITAN.NS" , "HCLTECH.NS","HAL.NS","PFC.NS",]    # Example stock
interval = "1d"        # Use "1m" for intraday
period="60d"   #"2d"  #2mo
# ✅ Fetch data
df = yf.download(nifty, period=period, interval=interval)
df.columns = df.columns.get_level_values(0)

# ✅ Timezone & Resampling
if interval == "1m":
    df.index = df.index.tz_convert("Asia/Kolkata")
    df["Date"] = df.index.normalize()  # keep datetime64[ns]
    df_3min = df.resample("3min").agg({
        "Open": "first",
        "High": "max",
        "Low": "min",
        "Close": "last",
        "Volume": "sum"
    }).dropna()
    df_3min["Date"] = df_3min.index.normalize()
else:
    df.index = pd.to_datetime(df.index)
    df["Date"] = df.index.normalize()
    df_3min = df.copy()

# ✅ Remove ambiguity by resetting index if needed
if 'Date' in df_3min.index.names:
    df_3min.reset_index(inplace=True, drop=True)

# ✅ Rounding and Indicators
df_3min["EMA_5"] = round(df_3min["Close"].ewm(span=5).mean())
df_3min["EMA_7"] = round(df_3min["Close"].ewm(span=7).mean())
df_3min["EMA_10"] = round(df_3min["Close"].ewm(span=10).mean())
df_3min["EMA_20"] = round(df_3min["Close"].ewm(span=20).mean())
df_3min["EMA_50"] = round(df_3min["Close"].ewm(span=50).mean())
df_3min["EMA_100"] = round(df_3min["Close"].ewm(span=100).mean())
df_3min["EMA_200"] = round(df_3min["Close"].ewm(span=200).mean())
df_3min["Pivot"] = (df_3min["High"] + df_3min["Low"] + df_3min["Close"]) / 3
df_3min["Close"] = round(df_3min["Close"])
df_3min["Open"] = round(df_3min["Open"])
df_3min["High"] = round(df_3min["High"])
df_3min["Low"] = round(df_3min["Low"])
df_3min["Volume"] = round(df_3min["Volume"])
df_3min["Prev_Vol"] = round(df_3min["Volume"].shift(1))
df_3min["Avg_volume"] = round(df_3min["Volume"].rolling(window=10).mean())

def classify_volume(row):
    if row["Volume"] > 1.5 * row["Avg_volume"]:
        return "HIGH_H"
    elif row["Volume"] > 2 * row["Avg_volume"]:
        return "HIGH_HH"
    elif row["Volume"] > row["Avg_volume"]:
        return "High"
    elif row["Volume"] < row["Avg_volume"]:
        return "LOW_Vol"
    else:
        return "Normal"


def is_level_broken(df, level, level_type='resistance', window=5):
    if level_type == 'resistance':
        # Check if any candle closes above the level
        return (df['Close'].iloc[-window:] > level).any()
    elif level_type == 'support':
        # Check if any candle closes below the level
        return (df['Close'].iloc[-window:] < level).any()
    return False

# Example check



df_3min["Vol_incr"] = df_3min.apply(classify_volume, axis=1)
df_3min['Up_Vol'] = df_3min['Volume'] > df_3min['Prev_Vol']
df_3min["Vol_incr"] = df_3min["Vol_incr"] +" todayvolIncr_" + df_3min["Up_Vol"].astype(str)

# ✅ Variance Calculation
df_3min["Variance"] = round(df_3min["Close"].rolling(window=10).var())
variance_threshold = round(df_3min["Variance"].mean() * 1.5, 2)
df_3min["Var_Th"] = variance_threshold


def swing_trading_signals(df):
    # Calculate required columns
    df['candle_size'] = df['Close'] - df['Open']
    df['avg_candle_size'] = df['candle_size'].rolling(window=20).mean()
    df['avg_volume_20'] = df['Volume'].rolling(window=20).mean()

    # Condition 1: Volume spike
    df['vol_spike'] = df['Volume'] > 1.5 * df['avg_volume_20']


    # Condition 2: Strong bullish candle
    df['bullish_body'] = df['Close'] > df['Open']
    df['strong_bullish_candle'] = df['candle_size'] > 1.2 * df['avg_candle_size']

    # Condition 3: Breakout above previous high
    df['breakout'] = df['Close'] > df['High'].shift(1)

    # Combine for buy signal
    df['buy_signal'] = df['vol_spike'] & df['bullish_body'] & df['strong_bullish_candle'] & df['breakout']

    # Swing low (for stop loss)
    df['swing_low'] = df['Low'].rolling(window=3, center=True).min()

    # Entry, Stop Loss, Target
    df['entry_price'] = df['Close'].where(df['buy_signal'])
    df['stop_loss'] = df['swing_low'].where(df['buy_signal'])
    df['risk'] = df['entry_price'] - df['stop_loss']
    df['target'] = df['entry_price'] + 1.5 * df['risk']

    return df

#"entry_price","target","risk","stop_loss"
#Important Entry and stop lost
df_3min = swing_trading_signals(df_3min)
buy_signals = df_3min[df_3min['buy_signal'] == True]




# ✅ Rolling Highs and Lows
df_3min["Max_High_5"] = df_3min["High"].rolling(window=5).max()
df_3min["Min_Low_5"] = df_3min["Low"].rolling(window=5).min()

# ✅ Previous Day High, Low, Volume
prev_highs = df_3min.groupby("Date")["High"].max().shift(1)
prev_lows = df_3min.groupby("Date")["Low"].min().shift(1)
prev_avg_vol = df_3min.groupby("Date")["Volume"].mean().shift(1)

# ✅ Map previous values to each row
df_3min["Prev_High"] = df_3min["Date"].map(prev_highs)
df_3min["Prev_Low"] = df_3min["Date"].map(prev_lows)
df_3min["Prev_Avg_Vol"] = df_3min["Date"].map(prev_avg_vol)


# ✅ ATR Calculation
def atr(df, window=10):
    high_low = df["High"] - df["Low"]
    high_close = np.abs(df["High"] - df["Close"].shift(1))
    low_close = np.abs(df["Low"] - df["Close"].shift(1))
    tr = np.maximum(high_low, np.maximum(high_close, low_close))
    return tr.rolling(window).mean()



# ✅ Supertrend Calculation
def supertrend(df, period=10, multiplier=3):
    df["ATR"] = atr(df, period)
    hl2 = (df["High"] + df["Low"]) / 2
    df["Upper_Band"] = hl2 + (multiplier * df["ATR"])
    df["Lower_Band"] = hl2 - (multiplier * df["ATR"])
    df["Supertrend"] = np.nan
    uptrend = True


    for i in range(1, len(df)):
        if df["Close"].iloc[i] > df["Upper_Band"].iloc[i-1]:
            uptrend = True
        elif df["Close"].iloc[i] < df["Lower_Band"].iloc[i-1]:
            uptrend = False

        df.loc[df.index[i], "Supertrend"] = df["Lower_Band"].iloc[i] if uptrend else df["Upper_Band"].iloc[i]

    return df

df_3min = supertrend(df_3min)
df_3min["ATR"] = atr(df_3min, 10)



lookback=14
# ✅ Compute RSI, MACD
df_3min["RSI"] = round(ta.momentum.rsi(df_3min["Close"], window=14))
df_3min["MACD"] = ta.trend.macd(df_3min["Close"])
df_3min["MACD_Signal"] = ta.trend.macd_signal(df_3min["Close"])

df_3min["HH"] = df_3min["High"].rolling(window=lookback).max()
df_3min["LL"] = df_3min["Low"].rolling(window=lookback).min()

# ✅ Compute Bollinger Bands
bb = BollingerBands(close=df_3min["Close"], window=20, window_dev=2)
df_3min["Upper_BB"] = bb.bollinger_hband()
df_3min["Lower_BB"] = bb.bollinger_lband()

df_3min['volatility'] = ta.volatility.bollinger_hband_indicator(df_3min['Close'], window=20)

# ✅ Compute ADX and Directional Indicators
adx = ADXIndicator(high=df_3min["High"], low=df_3min["Low"], close=df_3min["Close"], window=14)
df_3min["ADX"] = round(adx.adx())
df_3min["+DI"] = round( adx.adx_pos(),2)
df_3min["-DI"] =round(adx.adx_neg(),2)

# ✅ Detect ADX Increasing for the Last 5 Candles
df_3min["ADX_Incr"] = df_3min["ADX"].diff(5) > 0
df_3min["ADX_Decreasing"] = df_3min["ADX"].diff(5) < 0
df_3min['DI_Diff'] = abs(df_3min['+DI'] - df_3min['-DI'])  # Compute absolute difference

# ✅ Compute Pivot Points
df_3min["Pivot"] = round((df_3min["High"] + df_3min["Low"] + df_3min["Close"]) / 3,0)
df_3min["R1"] = round((2 * df_3min["Pivot"]) - df_3min["Low"], 0)
df_3min["S1"] = round((2 * df_3min["Pivot"]) - df_3min["High"], 0)
df_3min["R2"] = round(df_3min["Pivot"] + (df_3min["High"] - df_3min["Low"]), 0)
df_3min["S2"] = round(df_3min["Pivot"] - (df_3min["High"] - df_3min["Low"]), 0)
df_3min["Golden_Cross"] = (df_3min["EMA_20"] > df_3min["EMA_50"]) & (df_3min["EMA_20"].shift(1) < df_3min["EMA_50"].shift(1))
df_3min["Death_Cross"] = (df_3min["EMA_20"] < df_3min["EMA_50"]) & (df_3min["EMA_20"].shift(1) > df_3min["EMA_50"].shift(1))
df_3min["Golden/Death"] = np.where(df_3min["EMA_20"] > df_3min["EMA_50"],
                             np.where(df_3min["EMA_20"].shift(1) < df_3min["EMA_50"].shift(1), "BUY", "No Signal"),
                             np.where(df_3min["EMA_20"].shift(1) > df_3min["EMA_50"].shift(1), "SELL", "No Signal"))
df_3min["PriceAction-Buy"]=(df_3min["Close"] > df_3min["Open"]) & (df_3min["Open"].shift(1) > df_3min["Close"].shift(1)) & (df_3min["Close"] > df_3min["Open"].shift(1)) & (df_3min["Open"] < df_3min["Close"].shift(1))



df_3min["PriceAction_Sell"] = (
        (df_3min["Close"] < df_3min["Open"]) &
        (df_3min["Open"].shift(1) < df_3min["Close"].shift(1)) &
        (df_3min["Close"] < df_3min["Open"].shift(1)) &
        (df_3min["Open"] > df_3min["Close"].shift(1))
    )
df_3min["Doji"] = (
        abs(df_3min["Close"] - df_3min["Open"]) / df_3min["Open"] < 0.1
    )

df_3min["Price"] = "WAIT"

# Step 1: Detect Swing Highs and Lows
def detect_swing_highs_lows(df, lookback=2):
    df = df.copy()
    df['Swing_High'] = np.nan
    df['Swing_Low'] = np.nan

    for i in range(lookback, len(df) - lookback):
        window_highs = df['High'].iloc[i - lookback:i + lookback + 1]
        window_lows = df['Low'].iloc[i - lookback:i + lookback + 1]

        center = i

        if df['High'].iloc[center] == window_highs.max():
            df.at[df.index[center], 'Swing_High'] = round(df['High'].iloc[center], 2)

        if df['Low'].iloc[center] == window_lows.min():
            df.at[df.index[center], 'Swing_Low'] = round(df['Low'].iloc[center], 2)

    return df


# Step 2: Extract Support/Resistance Levels
def extract_sr_levels_from_swings(df, lookback=2, tolerance=0.5):
    swing_highs = df['Swing_High'].dropna().tail(lookback)
    swing_lows = df['Swing_Low'].dropna().tail(lookback)

    resistance_levels = []
    support_levels = []


    for price in swing_highs:
        if not any(abs(price - existing) <= tolerance for existing in resistance_levels):
            resistance_levels.append(price)

    for price in swing_lows:
        if not any(abs(price - existing) <= tolerance for existing in support_levels):
            support_levels.append(price)

    resistance_levels.sort(reverse=True)
    support_levels.sort()

    return resistance_levels, support_levels


# Step 3: Apply to your DataFrame (df_3min assumed to be your 3-minute interval data)
df_3min = detect_swing_highs_lows(df_3min, lookback=2)
res_levels, sup_levels = extract_sr_levels_from_swings(df_3min, lookback=2, tolerance=0.5)

# Step 4: Assign the most recent levels to the latest row
df_3min.loc[df_3min.index[-1], "res_level"] = res_levels[0] if res_levels else np.nan
df_3min.loc[df_3min.index[-1], "sup_level"] = sup_levels[0] if sup_levels else np.nan

df_3min["S-R"] = df_3min.apply(
    lambda row: f"{round(row['res_level'], 0)} - {round(row['sup_level'], 0)}"
    if pd.notna(row['res_level']) and pd.notna(row['sup_level']) else np.nan,
    axis=1
)

# # Set "BUY" signal where the PriceAction-Buy condition is True
df_3min.loc[df_3min["PriceAction-Buy"] == True, "Signal"] = "BUY"

# # Set "SELL" signal where the PriceAction-Sell condition is True, but only if "BUY" is not already set
# df_3min.loc[(df_3min["PriceAction_Sell"] == True) & (df_3min["Signal"] != "BUY"), "Signal"] = "SELL"

# # Set "WAIT" signal where Doji condition is True, but only if no other signal is present
df_3min.loc[(df_3min["Doji"] == True) & (df_3min["Signal"] == "WAIT"), "Signal"] = "WAIT"

def compute_heikin_ashi(df):
    ha_df = df.copy()
    ha_df['HA_Close'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4
    ha_df['HA_Open'] = (df['Open'].shift(1) + df['Close'].shift(1)) / 2

   # ha_df['HA_Open'].fillna((df['Open'] + df['Close']) / 2, inplace=True)
    ha_df['HA_Open'] = ha_df['HA_Open'].fillna((df['Open'] + df['Close']) / 2)
    ha_df['HA_High'] = ha_df[['High', 'HA_Open', 'HA_Close']].max(axis=1)
    ha_df['HA_Low'] = ha_df[['Low', 'HA_Open', 'HA_Close']].min(axis=1)
    return ha_df
def detect_engulfing_and_wicks(df):
    conditions = []

    for i in range(1, len(df)):
        row_prev = df.iloc[i - 1]
        row_curr = df.iloc[i]

        # Bullish Engulfing
        is_bullish_engulf = (
            row_prev['HA_Close'] < row_prev['HA_Open'] and
            row_curr['HA_Close'] > row_curr['HA_Open'] and
            row_curr['HA_Close'] > row_prev['HA_Open'] and
            row_curr['HA_Open'] < row_prev['HA_Close']
        )

        # Bearish Engulfing
        is_bearish_engulf = (
            row_prev['HA_Close'] > row_prev['HA_Open'] and
            row_curr['HA_Close'] < row_curr['HA_Open'] and
            row_curr['HA_Open'] > row_prev['HA_Close'] and
            row_curr['HA_Close'] < row_prev['HA_Open']
        )

        # Rejection wicks (Pin bar style)
        body = abs(row_curr['HA_Close'] - row_curr['HA_Open'])
        upper_wick = row_curr['HA_High'] - max(row_curr['HA_Close'], row_curr['HA_Open'])
        lower_wick = min(row_curr['HA_Close'], row_curr['HA_Open']) - row_curr['HA_Low']

        # Swing High pattern (rejection on top)
        swing_high = upper_wick > 2 * body and row_curr['HA_Close'] < row_curr['HA_Open']

        # Swing Low pattern (rejection at bottom)
        swing_low = lower_wick > 2 * body and row_curr['HA_Close'] > row_curr['HA_Open']

        conditions.append({
            'BullishEngulfing': is_bullish_engulf,
            'BearishEngulfing': is_bearish_engulf,
            'SwingHighWick': swing_high,
            'SwingLowWick': swing_low
        })

    return pd.DataFrame(conditions, index=df.index[1:])

def get_potential_swings(df):
    ha_df = compute_heikin_ashi(df)
    patterns = detect_engulfing_and_wicks(ha_df)

    ha_df = ha_df.iloc[1:]  # because patterns start from index 1
    ha_df = ha_df.join(patterns)

    ha_df['HA_SW_High'] = ha_df['BearishEngulfing'] | ha_df['SwingHighWick']
    ha_df['HA_SW_Low'] = ha_df['BullishEngulfing'] | ha_df['SwingLowWick']
    return ha_df
df_3min = get_potential_swings(df_3min)

def detect_heikin_ashi_trend(df, lookback=3):
    trends = []

    for i in range(len(df)):
        if i < lookback:
            trends.append(None)
            continue

        recent = df.iloc[i-lookback+1:i+1]

        # Count bullish/bearish candles
        bullish = (recent['HA_Close'] > recent['HA_Open']).sum()
        bearish = (recent['HA_Close'] < recent['HA_Open']).sum()

        if bullish == lookback:
            trends.append("Uptrend")
        elif bearish == lookback:
            trends.append("Downtrend")
        else:
            trends.append("Sideways")

    df["HA_Trend"] = trends
    return df
df_3min = detect_heikin_ashi_trend(df_3min)
df_3min["HA_Close"] = (df_3min["Open"] + df_3min["High"] + df_3min["Low"] + df_3min["Close"]) / 4
df_3min["HA_Open"] = df_3min["HA_Close"].shift(1)

df_3min["SMHT"] = (df_3min["MACD"] - df_3min["MACD"].rolling(window=10).mean()) / df_3min["MACD"].rolling(window=10).std()

# ✅ Buy/Sell Ranges
df_3min["Buy_Range"] = df_3min["S1"]  # Buying near S1
df_3min["Sell_Range"] = df_3min["R1"]  # Selling near R1

# Candle parameters
df_3min["Candle_Body"] = abs(df_3min["Close"] - df_3min["Open"])
df_3min["Prev_Candle_Body"] = df_3min["Candle_Body"].shift(1)
df_3min["Is_Bullish"] = df_3min["Close"] > df_3min["Open"]
df_3min["Is_Bearish"] = df_3min["Close"] < df_3min["Open"]


def generate_signals_with_dynamic_ranges(data):
    signals, scores, entries, targets, stops, rr_ratios = [], [], [], [], [], []

    for i in range(len(data)):
        row = data.iloc[i]
        close = row["Close"]
        open_ = row['Open']
        adx = row["ADX"]
        prev_row = data.iloc[i - 1]
        plus_di, minus_di = row["+DI"], row["-DI"]
        candle_body = row["Candle_Body"]
        prev_body = row["Prev_Candle_Body"]
        prev_close = prev_row["Close"]
        prev_high = prev_row["High"]
        prev_low = prev_row["Low"]
        variance = row["Variance"]
        supertrend = row["Supertrend"]
        ema5, ema10, ema20, ema50, ema200 = row["EMA_5"], row["EMA_10"], row["EMA_20"], row["EMA_50"], row["EMA_200"]
        upper_bb, lower_bb = row["Upper_BB"], row["Lower_BB"]
        adx_incr = row["ADX_Incr"]
        macd, macd_signal = row["MACD"], row["MACD_Signal"]
        smht = row["SMHT"]
        ha_close, ha_open = row["HA_Close"], row["HA_Open"]
        r1 = row.get("R1", None)
        s1 = row.get("S1", None)
        atr = row["ATR"]

        signal = "No Signal"
        entry = target = stop = None
        score = 0
               # --- Score Logic ---
        if adx > 25: score += 1 + (0.5 if adx_incr else 0)
        if plus_di > minus_di: score += 1
        if close > supertrend: score += 1.5
        if ema5 > ema10 > ema20: score += 1.5
        if macd > macd_signal: score += 1
        if variance > variance_threshold and close > upper_bb: score += 1.5
        if ema50 > ema200: score += 1
        if smht > 1 and ha_close > ha_open: score += 1.5
        if minus_di > plus_di: score -= 1
        if close < supertrend: score -= 1.5
        if ema5 < ema10 < ema20: score -= 1.5
        if macd < macd_signal: score -= 1
        if variance > variance_threshold and close < lower_bb: score -= 1.5
        if ema50 < ema200: score -= 1
        if smht < -1 and ha_close < ha_open: score -= 1.5

        # --- Signal Classification ---
        risk_buffer = close * 0.0075  # ~0.75% buffer
        vola_boost = variance * 0.1




        if score >= 4 :
            signal = f"Strong BUY ({round(score, 1)})"
            entry = close + close * 0.001  # buffer
            stop = lower_bb if lower_bb else (s1 if s1 else close - risk_buffer - vola_boost)
            target = upper_bb if upper_bb else (r1 if r1 else close + (close - stop) * 2)

        elif 2 <= score < 4:
            signal = f"BUY ({round(score, 1)})"
            entry = close + close * 0.0005
            stop = lower_bb if lower_bb else close - risk_buffer
            target = upper_bb if upper_bb else close + (close - stop) * 1.5

        elif score <= -4:
            signal = f"Strong SELL ({round(score, 1)})"
            entry = close # close - close * 0.001
            stop = upper_bb if upper_bb else (r1 if r1 else close + risk_buffer + vola_boost)
            if stop <= entry:
                risk = close * 0.01
                stop = entry + risk
            else:
                risk = stop - entry
            raw_target = entry - risk * 2
            target = lower_bb if lower_bb and lower_bb < entry else (
                s1 if s1 and s1 < entry else raw_target
            )

        elif -4 < score <= -2:
            signal = f"SELL ({round(score, 1)})"
            entry = close - close * 0.0005
            stop = upper_bb if upper_bb else (r1 if r1 else close + risk_buffer + vola_boost)
            if stop <= entry:
                risk = close * 0.01
                stop = entry + risk
            else:
                risk = stop - entry
            raw_target = entry - risk * 2
            target = lower_bb if lower_bb and lower_bb < entry else (
                s1 if s1 and s1 < entry else raw_target
            )



        # Reward/Risk
        rr = round((target - entry) / (entry - stop), 2) if entry and stop and target else ""



        signals.append(signal)
        scores.append(score)
        entries.append(round(entry, 2) if entry else "")
        targets.append(round(target, 2) if target else "")
        stops.append(round(stop, 2) if stop else "")
        rr_ratios.append(rr)

    return signals, scores, entries, targets, stops, rr_ratios

current_close = df_3min['Close'].iloc[-1]
current_high = df_3min['High'].iloc[-1]
current_low = df_3min['Low'].iloc[-1]

# 🔹 Get recent valid swing points (last N)
recent_swing_highs = df_3min['Swing_High'].dropna().tail(5)
recent_swing_lows = df_3min['Swing_Low'].dropna().tail(5)

def generate_smht_signal(df):
    signals = []
    for smht in df["SMHT"]:
        if smht > 2:
            signals.append("Strong Buy")
        elif smht < -2:
            signals.append("Strong Sell")
        else:
            signals.append("")
    return signals

# Usage:
df_3min["SMHT_Signal"] = generate_smht_signal(df_3min)

df_3min["HA_SW_High"] = df_3min["HA_SW_High"].apply(lambda x: "T" if x else "F")
df_3min["HA_SW_Low"]= df_3min["HA_SW_Low"].apply(lambda x: "T" if x else "F")
df_3min["Signal"], df_3min["Score"], df_3min["Entry"], df_3min["Target"], df_3min["StopLoss"], df_3min["RR"] = generate_signals_with_dynamic_ranges(df_3min)
#df_3min["EMA_20>50"] = np.where(df_3min["EMA_20"] > df_3min["EMA_50"], "T", "F")
df_3min["EMA_Status"] = (
    np.where(df_3min["Close"] > df_3min["EMA_5"], "T", "F") +
    np.where(df_3min["EMA_5"] > df_3min["EMA_10"], "T", "F") +
    np.where(df_3min["EMA_10"] > df_3min["EMA_20"], "T", "F") +
    np.where(df_3min["EMA_20"] > df_3min["EMA_50"], "T", "F") +
    np.where(df_3min["EMA_50"] > df_3min["EMA_100"], "T", "F")
)
Entry_condition = (
    (df_3min["EMA_Status"] == "TTTT") &
    (df_3min["HA_Trend"] == "Uptrend") &
    --(df_3min["MACD"] > df_3min["MACD_Signal"]) &
    (df_3min["ADX"] > 20) &
    (df_3min["+DI"] > df_3min["-DI"])
)

Exit_condition = (
    (df_3min["EMA_Status"] == "FFFF") &
    (df_3min["HA_Trend"] == "Downtrend") &
    (df_3min["MACD"] < df_3min["MACD_Signal"]) &
    (df_3min["ADX"] > 20) &
    (df_3min["+DI"] < df_3min["-DI"])
)
df_3min["EE_Signal"] = np.where(Entry_condition, "Buy",
                       np.where(Exit_condition, "Sell", ""))


df_3min["Yest_H/L"]=""
#Based on the Swing high and low giving the signals
# ✅ Add yesterday's high and low columns
df_3min['Prev_High'] = df_3min['High'].shift(1)
df_3min['Prev_Low'] = df_3min['Low'].shift(1)
df_3min['Prev_Close'] = df_3min['Close'].shift(1)

df_3min['Prev_High_Fill'] = df_3min['Swing_High'].ffill()
df_3min['Prev_Low_Fill'] = df_3min['Swing_Low'].ffill()

# Breakout conditions
df_3min['Break_High'] = (df_3min['Close'] > df_3min['Prev_High_Fill']) & (df_3min['Close'] > df_3min['Prev_Low_Fill'])
df_3min['Break_Low'] = (df_3min['Close'] < df_3min['Prev_Low_Fill']) & (df_3min['Close'] < df_3min['Prev_High_Fill'])
df_3min['Break_Prevs_SwingLow'] = (df_3min['Close'] > df_3min['Prev_Low_Fill']) & (df_3min['Close'] < df_3min['Prev_High_Fill'])
df_3min['Body'] = abs(df_3min['Close'] - df_3min['Open'])
df_3min['Strong_Break_High'] = df_3min['Break_High'] & (df_3min['Body'] > df_3min['ATR'] * 0.8)
atr_multiplier = 1.5
df_3min['ATR_Break_High'] = df_3min['Close'] > (df_3min['Prev_High_Fill'] + atr_multiplier * df_3min['ATR'])
df_3min['ATR_Break_Low'] = df_3min['Close'] < (df_3min['Prev_Low_Fill'] - atr_multiplier * df_3min['ATR'])
df_3min['Breakout_with_Momentum'] = (df_3min['Break_High']) & (df_3min['RSI'] > 60)
df_3min['Inside_Bar'] = (df_3min['High'] < df_3min['Prev_High']) & (df_3min['Low'] > df_3min['Prev_Low'])

conditions =[
    df_3min['Break_High'],
    df_3min['Break_Low'],
    df_3min['Break_Prevs_SwingLow'],
    df_3min['Strong_Break_High'],
    df_3min['ATR_Break_High'],
    df_3min['ATR_Break_Low'],
    df_3min['Breakout_with_Momentum']
    ]


choices = ['SwingHIGH-Breakout-BUY', 'SWingLow-Breakdown-SELL','Swinglow_Breakout-ShortBuy','Strong_Break_High-BUY','ATR_Break_High-BUY','ATR_Break_Low-SELL','Breakout_with_Momentum-StrongBUY']

df_3min['Swing_Signal'] = np.select(conditions, choices, default='Hold')+"-" +df_3min["EMA_Status"]

# Step 2: Round swing levels to cluster nearby values
df_3min['resistance_rounded'] = df_3min['Swing_High'].round(-1)   # Round to nearest 10
df_3min['support_rounded']    = df_3min['Swing_Low'].round(-1)

# Step 3: Count frequency of each level
resistance_levels = df_3min['resistance_rounded'].value_counts().sort_values(ascending=False)
support_levels    = df_3min['support_rounded'].value_counts().sort_values(ascending=False)

# Filter strong levels (e.g., appeared at least 2 times)
#strong Resistance and suuport
strong_resistances = resistance_levels[resistance_levels >= 2]
strong_supports    = support_levels[support_levels >= 2]

# strong Resistance and suuport


# ✅ Check for breakouts based on yesterday's high and low
df_3min['Yesterday_High_Breaks_Close'] = df_3min['Close'] >df_3min['Prev_High']
df_3min['Yesterday_Low_Breaks_Close'] = df_3min['Close'] < df_3min['Prev_Low']
df_3min['Yesterday_Low_Breaks_Low'] = df_3min['Low'] < df_3min['Prev_Low']
df_3min['Yesterday_High_Beaks_PrevHigh'] = df_3min['High'] > df_3min['Prev_High']
df_3min['Yesterday_Low_Beaks_PrevHigh'] = df_3min['Low'] > df_3min['Prev_High']
df_3min['Today_High_Breaks_SwingHigh'] = df_3min['Close'] > df_3min['Prev_High_Fill']
df_3min['Today_High_Breaks_SwingLow'] = df_3min['Low'] < df_3min['Prev_Low_Fill']
df_3min['Today_CloseBreaks_BB_Upper'] = df_3min['Close'] > df_3min['Upper_BB']
df_3min['Today_LowBreakdown_BB_Lower'] = df_3min['Low'] < df_3min['Lower_BB']

df_3min['Yest_H/L'] = np.where(df_3min['Yesterday_High_Breaks_Close'], 'Current Close breaks Previous High- BUY',
                             np.where(df_3min['Yesterday_Low_Breaks_Close'], 'Current Close lessthan Previous Low -Sell',
                              np.where(df_3min['Today_High_Breaks_SwingHigh'], 'Current Close breaks Swingh High -BUY',
                               np.where(df_3min['Today_High_Breaks_SwingLow'], 'Current Close breakdown Swing low -Sell',
                                  np.where(df_3min['Yesterday_High_Beaks_PrevHigh'], 'Current High Breakout PrevHigh -BUY',
                                         np.where(df_3min['Yesterday_Low_Beaks_PrevHigh'], 'Current Low Breakout PrevHigh -BUY',
                                                  np.where(df_3min['Today_CloseBreaks_BB_Upper'], 'Current Close Breakout BB_UPPER -BUY',
                                                             np.where(df_3min['Today_LowBreakdown_BB_Lower'], 'Current Low Breakdown BB_Lower -SELL',
                             np.where(df_3min['Yesterday_Low_Breaks_Low'], 'Current Low lessthan Previous Low -Sell', 'No Break')))))))))


df_3min["strong_buy"] = (
    (df_3min["EMA_5"] > df_3min["EMA_10"]) &
    (df_3min["EMA_10"] > df_3min["EMA_20"]) &
    (df_3min["ADX"] > 25) &
    (df_3min["+DI"] > df_3min["-DI"]) &
    (df_3min["Supertrend"] < df_3min["Close"]) &
    #(df_3min["RSI"] < 60) &
    (df_3min["HA_SW_Low"] == "T") &
    (df_3min["Close"] > df_3min["S1"])  # Confirmed above support

    )
df_3min["strong_sell"] = (
    (df_3min["EMA_5"] < df_3min["EMA_10"]) &
    (df_3min["EMA_10"] < df_3min["EMA_20"]) &
    (df_3min["ADX"] > 25) &
    (df_3min["-DI"] > df_3min["+DI"]) &
    (df_3min["Supertrend"] > df_3min["Close"]) &
    #(df_3min["RSI"] > 40) &
    (df_3min["HA_SW_High"] == "T") &
    (df_3min["Close"] < df_3min["R1"])  # Rejected at resistance
)
df_3min["Final_Signal"] = np.where(
    df_3min["strong_buy"], "STRONG BUY",
    np.where(df_3min["strong_sell"], "STRONG SELL", df_3min["Yest_H/L"])
    )
# Use previous row's S1 and R1
df_3min["S1_prev"] = df_3min["S1"].shift(1)
df_3min["R1_prev"] = df_3min["R1"].shift(1)
# Forward fill swing highs for reference
df_3min['Last_Swing_High'] = df_3min['Swing_High'].ffill()
df_3min['Last_Swing_Low'] = df_3min['Swing_Low'].ffill()

# Detect price breakout above resistance (last 20-day high)
df_3min['Resistance'] = df_3min['High'].rolling(window=20).max().shift(1)
df_3min['Price_Breakout'] = df_3min['Close'] > df_3min['Resistance']

# Detect breakout of previous swing high
df_3min['Swing_High_Breakout'] = df_3min['Close'] > df_3min['Last_Swing_High']
df_3min['Volume_SMA20'] = df_3min['Volume'].rolling(window=20).mean()

df_3min['Support'] = df_3min['Low'].rolling(window=20).min().shift(1)
df_3min['Price_Breakdown'] = df_3min['Close'] < df_3min['Support']
df_3min['Swing_Low_Breakdown'] = df_3min['Close'] < df_3min['Last_Swing_Low']

# Volume confirmation
df_3min['Volume_Confirm'] = df_3min['Volume'] > df_3min['Volume_SMA20']

# Signal Logic
def get_signal(row):
    if row['Price_Breakout'] and row['Volume_Confirm']:
        return "Strong Buy (Price Breakout)"
    elif row['Swing_High_Breakout'] and row['Volume_Confirm']:
        return "Buy (Swing High Breakout)"
    elif row['Price_Breakout'] or row['Swing_High_Breakout']:
        return "Watch (No Volume Confirmation)"

       # Sell Side
    elif row['Price_Breakdown'] and row['Volume_Confirm']:
        return "Strong Sell (Price Breakdown)"
    elif row['Swing_Low_Breakdown'] and row['Volume_Confirm']:
        return "Sell (Swing Low Breakdown)"
    elif row['Price_Breakdown'] or row['Swing_Low_Breakdown']:
        return "Watch (Possible Sell, No Volume Confirmation)"
    return ""

df_3min['LongBuy_Signal'] = df_3min.apply(get_signal, axis=1)
df_3min['buy_signal'] = df_3min['buy_signal'].astype(str) + df_3min['LongBuy_Signal'].astype(str)

#df_today=df_3min[["Close","High","Low", "+DI", "-DI","DI_Diff","Variance", "S1", "R1", "S2","R2","HA_Signal", "ADX", "ADX_Incr", "Signal", "EMA_5", "EMA_10", "EMA_20","Golden/Death"]]
#df_3min=df_3min[["Open","Close","High","Low", "Signal", "Vol_incr","S1","R1","S2","R2","entry_price","target","risk","stop_loss"]]

df_3min=df_3min[["Date","Open","Close","High","Low","Signal","S1","R1","Vol_incr","Volume","buy_signal","Swing_Signal","Swing_High","Swing_Low","ADX","+DI","-DI","HA_SW_High","HA_SW_Low","HA_Trend"]]
# filename = f"nifty50BB_{nifty}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
# df_3min.to_csv(filename)
#df_3min = df_3min[(df_3min["EE_Signal"] == "Buy") & (df_3min["HA_SW_Low"] == "T")] # for buy
#df_3min = df_3min[(df_3min["EE_Signal"] == "Buy")] # for buy
#df_3min = df_3min[(df_3min["HA_SW_Low"] == "T")] # for buy
#df_3min = df_3min[(df_3min["HA_SW_High"] == "T")] # for sell
df_3min.tail(10)



[*********************100%***********************]  1 of 1 completed


,Date,Open,Close,High,Low,Signal,S1,R1,Vol_incr,Volume,buy_signal,Swing_Signal,Swing_High,Swing_Low,ADX,+DI,-DI,HA_SW_High,HA_SW_Low,HA_Trend
50,2025-05-05,1828.0,1837.0,1843.0,1821.0,Strong BUY (7.5),1825.0,1847.0,LOW_Vol todayvolIncr_False,1219830,False,Swinglow_Breakout-ShortBuy-TTTTT,NaN,NaN,23.0,27.44,13.24,F,F,Sideways
51,2025-05-06,1820.0,1819.0,1828.0,1799.0,Strong BUY (6.0),1802.0,1831.0,LOW_Vol todayvolIncr_True,1414772,False,Swinglow_Breakout-ShortBuy-FTTTT,NaN,NaN,23.0,25.78,15.94,T,F,Sideways
52,2025-05-07,1818.0,1781.0,1826.0,1779.0,Strong BUY (6.0),1764.0,1811.0,LOW_Vol todayvolIncr_True,2015237,False,Swinglow_Breakout-ShortBuy-FTTTT,NaN,NaN,23.0,23.86,17.92,F,F,Sideways
53,2025-05-08,1781.0,1767.0,1789.0,1755.0,BUY (2.5),1751.0,1785.0,High todayvolIncr_True,2647871,False,Swinglow_Breakout-ShortBuy-FFTTT,NaN,NaN,21.0,22.55,20.81,F,F,Downtrend
54,2025-05-09,1752.0,1745.0,1766.0,1732.0,No Signal,1730.0,1764.0,LOW_Vol todayvolIncr_False,1148956,False,Swinglow_Breakout-ShortBuy-FFTTT,NaN,NaN,20.0,21.26,23.38,F,F,Downtrend
55,2025-05-12,1696.0,1686.0,1712.0,1624.0,No Signal,1636.0,1724.0,HIGH_H todayvolIncr_True,9221075,False,Swinglow_Breakout-ShortBuy-FFTTT,NaN,1624.0,21.0,17.52,34.97,F,F,Downtrend
56,2025-05-13,1711.0,1700.0,1748.0,1691.0,No Signal,1678.0,1735.0,HIGH_H todayvolIncr_False,6988329,False,Swinglow_Breakout-ShortBuy-FFFTT,NaN,NaN,21.0,21.11,31.87,F,F,Sideways
57,2025-05-14,1702.0,1708.0,1711.0,1692.0,SELL (-2.5),1697.0,1716.0,LOW_Vol todayvolIncr_False,1639324,False,Swinglow_Breakout-ShortBuy-FFFTT,NaN,NaN,21.0,20.51,30.97,T,F,Sideways
58,2025-05-15,1701.0,1740.0,1744.0,1684.0,No Signal,1702.0,1762.0,High todayvolIncr_True,3880385,False,Swinglow_Breakout-ShortBuy-TFFTT,NaN,NaN,20.0,23.54,28.25,F,F,Sideways
59,2025-05-16,1740.0,1735.0,1742.0,1719.0,No Signal,1722.0,1745.0,LOW_Vol todayvolIncr_False,1448249,False,Swinglow_Breakout-ShortBuy-TFFTT,NaN,NaN,19.0,22.72,27.26,F,F,Sideways


In [46]:
# @title Day Trading-Options or Stocks
#Day Trading or Options trading
import yfinance as yf
import pandas as pd
import numpy as np
from ta.trend import ADXIndicator
from ta.volatility import BollingerBands
from ta.volatility import AverageTrueRange

# ✅ Symbol and Interval
nifty =   "^NSEI"     # "^NSEI"     ^NSEBANK   # Example stock
nifty_S =  ["TITAN.NS" , "HCLTECH.NS","HAL.NS","PFC.NS"]    # Example stock
interval = "1m"        # Use "1m" for intraday
period="2d"   #"2d"  #2mo
# ✅ Fetch data
df = yf.download(nifty, period=period, interval=interval)
df.columns = df.columns.get_level_values(0)

# ✅ Timezone & Resampling
if interval == "1m":
    df.index = df.index.tz_convert("Asia/Kolkata")
    df["Date"] = df.index.normalize()  # keep datetime64[ns]
    df_3min = df.resample("2min").agg({
        "Open": "first",
        "High": "max",
        "Low": "min",
        "Close": "last",
        "Volume": "sum"
    }).dropna()
    df_3min["Date"] = df_3min.index.normalize()
else:
    df.index = pd.to_datetime(df.index)
    df["Date"] = df.index.normalize()
    df_3min = df.copy()

# ✅ Remove ambiguity by resetting index if needed
if 'Date' in df_3min.index.names:
    df_3min.reset_index(inplace=True, drop=True)

# ✅ Rounding and Indicators
df_3min["EMA_5"] = round(df_3min["Close"].ewm(span=5).mean())
df_3min["EMA_7"] = round(df_3min["Close"].ewm(span=7).mean())
df_3min["EMA_10"] = round(df_3min["Close"].ewm(span=10).mean())
df_3min["EMA_20"] = round(df_3min["Close"].ewm(span=20).mean())
df_3min["EMA_50"] = round(df_3min["Close"].ewm(span=50).mean())
df_3min["EMA_100"] = round(df_3min["Close"].ewm(span=100).mean())
df_3min["EMA_200"] = round(df_3min["Close"].ewm(span=200).mean())
df_3min["Pivot"] = (df_3min["High"] + df_3min["Low"] + df_3min["Close"]) / 3
df_3min["Close"] = round(df_3min["Close"])
df_3min["Open"] = round(df_3min["Open"])
df_3min["High"] = round(df_3min["High"])
df_3min["Low"] = round(df_3min["Low"])
df_3min["Volume"] = round(df_3min["Volume"])
df_3min["Avg_volume"] = round(df_3min["Volume"].rolling(window=10).mean())

def classify_volume(row):
    if row["Volume"] > 1.5 * row["Avg_volume"]:
        return "HIGH_H"
    elif row["Volume"] > 2 * row["Avg_volume"]:
        return "HIGH_HH"
    elif row["Volume"] > row["Avg_volume"]:
        return "High"
    else:
        return "Normal"

df_3min["Vol_incr"] = df_3min.apply(classify_volume, axis=1)
# ✅ Variance Calculation
df_3min["Variance"] = round(df_3min["Close"].rolling(window=10).var())
variance_threshold = round(df_3min["Variance"].mean() * 1.5, 2)
df_3min["Var_Th"] = variance_threshold

# ✅ Rolling Highs and Lows
df_3min["Max_High_5"] = df_3min["High"].rolling(window=5).max()
df_3min["Min_Low_5"] = df_3min["Low"].rolling(window=5).min()

# ✅ Previous Day High, Low, Volume
prev_highs = df_3min.groupby("Date")["High"].max().shift(1)
prev_lows = df_3min.groupby("Date")["Low"].min().shift(1)
prev_avg_vol = df_3min.groupby("Date")["Volume"].mean().shift(1)

# ✅ Map previous values to each row
df_3min["Prev_High"] = df_3min["Date"].map(prev_highs)
df_3min["Prev_Low"] = df_3min["Date"].map(prev_lows)
df_3min["Prev_Avg_Vol"] = df_3min["Date"].map(prev_avg_vol)


# ✅ ATR Calculation
def atr(df, period=10):
    high_low = df["High"] - df["Low"]
    high_close = np.abs(df["High"] - df["Close"].shift(1))
    low_close = np.abs(df["Low"] - df["Close"].shift(1))
    tr = np.maximum(high_low, np.maximum(high_close, low_close))
    return tr.rolling(period).mean()

# ✅ Supertrend Calculation
def supertrend(df, period=10, multiplier=3):
    df["ATR"] = atr(df, period)
    hl2 = (df["High"] + df["Low"]) / 2
    df["Upper_Band"] = hl2 + (multiplier * df["ATR"])
    df["Lower_Band"] = hl2 - (multiplier * df["ATR"])
    df["Supertrend"] = np.nan
    uptrend = True


    for i in range(1, len(df)):
        if df["Close"].iloc[i] > df["Upper_Band"].iloc[i-1]:
            uptrend = True
        elif df["Close"].iloc[i] < df["Lower_Band"].iloc[i-1]:
            uptrend = False

        df.loc[df.index[i], "Supertrend"] = df["Lower_Band"].iloc[i] if uptrend else df["Upper_Band"].iloc[i]

    return df

df_3min = supertrend(df_3min)



lookback=14
# ✅ Compute RSI, MACD
df_3min["RSI"] = round(ta.momentum.rsi(df_3min["Close"], window=14))
df_3min["MACD"] = ta.trend.macd(df_3min["Close"])
df_3min["MACD_Signal"] = ta.trend.macd_signal(df_3min["Close"])

df_3min["HH"] = df_3min["High"].rolling(window=lookback).max()
df_3min["LL"] = df_3min["Low"].rolling(window=lookback).min()

# ✅ Compute Bollinger Bands
bb = BollingerBands(close=df_3min["Close"], window=20, window_dev=2)
df_3min["Upper_BB"] = bb.bollinger_hband()
df_3min["Lower_BB"] = bb.bollinger_lband()

df_3min['volatility'] = ta.volatility.bollinger_hband_indicator(df_3min['Close'], window=20)

# ✅ Compute ADX and Directional Indicators
adx = ADXIndicator(high=df_3min["High"], low=df_3min["Low"], close=df_3min["Close"], window=14)
df_3min["ADX"] = round(adx.adx())
df_3min["+DI"] = round( adx.adx_pos(),2)
df_3min["-DI"] =round(adx.adx_neg(),2)

# ✅ Detect ADX Increasing for the Last 5 Candles
df_3min["ADX_Incr"] = df_3min["ADX"].diff(5) > 0
df_3min["ADX_Decreasing"] = df_3min["ADX"].diff(5) < 0
df_3min['DI_Diff'] = abs(df_3min['+DI'] - df_3min['-DI'])  # Compute absolute difference

# ✅ Compute Pivot Points
df_3min["Pivot"] = round((df_3min["High"] + df_3min["Low"] + df_3min["Close"]) / 3,0)
df_3min["R1"] = round((2 * df_3min["Pivot"]) - df_3min["Low"], 0)
df_3min["S1"] = round((2 * df_3min["Pivot"]) - df_3min["High"], 0)
df_3min["R2"] = round(df_3min["Pivot"] + (df_3min["High"] - df_3min["Low"]), 0)
df_3min["S2"] = round(df_3min["Pivot"] - (df_3min["High"] - df_3min["Low"]), 0)
df_3min["Golden_Cross"] = (df_3min["EMA_20"] > df_3min["EMA_50"]) & (df_3min["EMA_20"].shift(1) < df_3min["EMA_50"].shift(1))
df_3min["Death_Cross"] = (df_3min["EMA_20"] < df_3min["EMA_50"]) & (df_3min["EMA_20"].shift(1) > df_3min["EMA_50"].shift(1))
df_3min["Golden/Death"] = np.where(df_3min["EMA_20"] > df_3min["EMA_50"],
                             np.where(df_3min["EMA_20"].shift(1) < df_3min["EMA_50"].shift(1), "BUY", "No Signal"),
                             np.where(df_3min["EMA_20"].shift(1) > df_3min["EMA_50"].shift(1), "SELL", "No Signal"))
df_3min["PriceAction-Buy"]=(df_3min["Close"] > df_3min["Open"]) & (df_3min["Open"].shift(1) > df_3min["Close"].shift(1)) & (df_3min["Close"] > df_3min["Open"].shift(1)) & (df_3min["Open"] < df_3min["Close"].shift(1))



df_3min["PriceAction_Sell"] = (
        (df_3min["Close"] < df_3min["Open"]) &
        (df_3min["Open"].shift(1) < df_3min["Close"].shift(1)) &
        (df_3min["Close"] < df_3min["Open"].shift(1)) &
        (df_3min["Open"] > df_3min["Close"].shift(1))
    )
df_3min["Doji"] = (
        abs(df_3min["Close"] - df_3min["Open"]) / df_3min["Open"] < 0.1
    )

df_3min["Price"] = "WAIT"

#Japanise candel sticks
# Step 2: Define candlestick patterns
def bullish_engulfing(df):
    return (
        (df['Close'].shift(1) < df['Open'].shift(1)) &
        (df['Close'] > df['Open']) &
        (df['Open'] < df['Close'].shift(1)) &
        (df['Close'] > df['Open'].shift(1))
    )

def bearish_engulfing(df):
    return (
        (df['Close'].shift(1) > df['Open'].shift(1)) &
        (df['Close'] < df['Open']) &
        (df['Open'] > df['Close'].shift(1)) &
        (df['Close'] < df['Open'].shift(1))
    )

# Step 3: Generate signals
df_3min['JP_Buy_Signal'] = bullish_engulfing(df_3min)
df_3min['JP_Sell_Signal'] = bearish_engulfing(df_3min)

# Step 2: Generate combined signals using np.where
df_3min['JP_Signal'] = np.where(
    bullish_engulfing(df_3min), "BUY",
    np.where(bearish_engulfing(df_3min), "SELL", "NEU")
)







# Step 1: Detect Swing Highs and Lows
def detect_swing_highs_lows(df, lookback=2):
    df = df.copy()
    df['Swing_High'] = np.nan
    df['Swing_Low'] = np.nan

    for i in range(lookback, len(df) - lookback):
        window_highs = df['High'].iloc[i - lookback:i + lookback + 1]
        window_lows = df['Low'].iloc[i - lookback:i + lookback + 1]

        center = i

        if df['High'].iloc[center] == window_highs.max():
            df.at[df.index[center], 'Swing_High'] = round(df['High'].iloc[center], 2)

        if df['Low'].iloc[center] == window_lows.min():
            df.at[df.index[center], 'Swing_Low'] = round(df['Low'].iloc[center], 2)

    return df





# Step 3: Apply to your DataFrame (df_3min assumed to be your 3-minute interval data)
df_3min = detect_swing_highs_lows(df_3min, lookback=2)


# # Set "BUY" signal where the PriceAction-Buy condition is True
df_3min.loc[df_3min["PriceAction-Buy"] == True, "Signal"] = "BUY"

# # Set "SELL" signal where the PriceAction-Sell condition is True, but only if "BUY" is not already set
# df_3min.loc[(df_3min["PriceAction_Sell"] == True) & (df_3min["Signal"] != "BUY"), "Signal"] = "SELL"

# # Set "WAIT" signal where Doji condition is True, but only if no other signal is present
df_3min.loc[(df_3min["Doji"] == True) & (df_3min["Signal"] == "WAIT"), "Signal"] = "WAIT"

def compute_heikin_ashi(df):
    ha_df = df.copy()
    ha_df['HA_Close'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4
    ha_df['HA_Open'] = (df['Open'].shift(1) + df['Close'].shift(1)) / 2

   # ha_df['HA_Open'].fillna((df['Open'] + df['Close']) / 2, inplace=True)
    ha_df['HA_Open'] = ha_df['HA_Open'].fillna((df['Open'] + df['Close']) / 2)
    ha_df['HA_High'] = ha_df[['High', 'HA_Open', 'HA_Close']].max(axis=1)
    ha_df['HA_Low'] = ha_df[['Low', 'HA_Open', 'HA_Close']].min(axis=1)
    return ha_df
def detect_engulfing_and_wicks(df):
    conditions = []

    for i in range(1, len(df)):
        row_prev = df.iloc[i - 1]
        row_curr = df.iloc[i]

        # Bullish Engulfing
        is_bullish_engulf = (
            row_prev['HA_Close'] < row_prev['HA_Open'] and
            row_curr['HA_Close'] > row_curr['HA_Open'] and
            row_curr['HA_Close'] > row_prev['HA_Open'] and
            row_curr['HA_Open'] < row_prev['HA_Close']
        )

        # Bearish Engulfing
        is_bearish_engulf = (
            row_prev['HA_Close'] > row_prev['HA_Open'] and
            row_curr['HA_Close'] < row_curr['HA_Open'] and
            row_curr['HA_Open'] > row_prev['HA_Close'] and
            row_curr['HA_Close'] < row_prev['HA_Open']
        )

        # Rejection wicks (Pin bar style)
        body = abs(row_curr['HA_Close'] - row_curr['HA_Open'])
        upper_wick = row_curr['HA_High'] - max(row_curr['HA_Close'], row_curr['HA_Open'])
        lower_wick = min(row_curr['HA_Close'], row_curr['HA_Open']) - row_curr['HA_Low']

        # Swing High pattern (rejection on top)
        swing_high = upper_wick > 2 * body and row_curr['HA_Close'] < row_curr['HA_Open']

        # Swing Low pattern (rejection at bottom)
        swing_low = lower_wick > 2 * body and row_curr['HA_Close'] > row_curr['HA_Open']

        conditions.append({
            'BullishEngulfing': is_bullish_engulf,
            'BearishEngulfing': is_bearish_engulf,
            'SwingHighWick': swing_high,
            'SwingLowWick': swing_low
        })

    return pd.DataFrame(conditions, index=df.index[1:])

def get_potential_swings(df):
    ha_df = compute_heikin_ashi(df)
    patterns = detect_engulfing_and_wicks(ha_df)

    ha_df = ha_df.iloc[1:]  # because patterns start from index 1
    ha_df = ha_df.join(patterns)

    ha_df['HA_SW_High'] = ha_df['BearishEngulfing'] | ha_df['SwingHighWick']
    ha_df['HA_SW_Low'] = ha_df['BullishEngulfing'] | ha_df['SwingLowWick']
    return ha_df
df_3min = get_potential_swings(df_3min)

def detect_heikin_ashi_trend(df, lookback=3):
    trends = []

    for i in range(len(df)):
        if i < lookback:
            trends.append(None)
            continue

        recent = df.iloc[i-lookback+1:i+1]

        # Count bullish/bearish candles
        bullish = (recent['HA_Close'] > recent['HA_Open']).sum()
        bearish = (recent['HA_Close'] < recent['HA_Open']).sum()

        if bullish == lookback:
            trends.append("Uptrend")
        elif bearish == lookback:
            trends.append("Downtrend")
        else:
            trends.append("Sideways")

    df["HA_Trend"] = trends
    return df
df_3min = detect_heikin_ashi_trend(df_3min)
df_3min["HA_Close"] = (df_3min["Open"] + df_3min["High"] + df_3min["Low"] + df_3min["Close"]) / 4
df_3min["HA_Open"] = df_3min["HA_Close"].shift(1)

df_3min["SMHT"] = (df_3min["MACD"] - df_3min["MACD"].rolling(window=10).mean()) / df_3min["MACD"].rolling(window=10).std()

# ✅ Buy/Sell Ranges
df_3min["Buy_Range"] = df_3min["S1"]  # Buying near S1
df_3min["Sell_Range"] = df_3min["R1"]  # Selling near R1

# Candle parameters
df_3min["Candle_Body"] = abs(df_3min["Close"] - df_3min["Open"])
df_3min["Prev_Candle_Body"] = df_3min["Candle_Body"].shift(1)
df_3min["Is_Bullish"] = df_3min["Close"] > df_3min["Open"]
df_3min["Is_Bearish"] = df_3min["Close"] < df_3min["Open"]


def generate_signals_with_dynamic_ranges(data):
    signals, scores, entries, targets, stops, rr_ratios = [], [], [], [], [], []

    for i in range(len(data)):
        row = data.iloc[i]
        close = row["Close"]
        open_ = row['Open']
        adx = row["ADX"]
        prev_row = data.iloc[i - 1]
        plus_di, minus_di = row["+DI"], row["-DI"]
        candle_body = row["Candle_Body"]
        prev_body = row["Prev_Candle_Body"]
        prev_close = prev_row["Close"]
        prev_high = prev_row["High"]
        prev_low = prev_row["Low"]
        variance = row["Variance"]
        supertrend = row["Supertrend"]
        ema5, ema10, ema20, ema50, ema200 = row["EMA_5"], row["EMA_10"], row["EMA_20"], row["EMA_50"], row["EMA_200"]
        upper_bb, lower_bb = row["Upper_BB"], row["Lower_BB"]
        adx_incr = row["ADX_Incr"]
        macd, macd_signal = row["MACD"], row["MACD_Signal"]
        smht = row["SMHT"]
        ha_close, ha_open = row["HA_Close"], row["HA_Open"]
        r1 = row.get("R1", None)
        s1 = row.get("S1", None)
        atr = row["ATR"]

        signal = "No Signal"
        entry = target = stop = None
        score = 0
               # --- Score Logic ---
        if adx > 25: score += 1 + (0.5 if adx_incr else 0)
        if plus_di > minus_di: score += 1
        if close > supertrend: score += 1.5
        if ema5 > ema10 > ema20: score += 1.5
        if macd > macd_signal: score += 1
        if variance > variance_threshold and close > upper_bb: score += 1.5
        if ema50 > ema200: score += 1
        if smht > 1 and ha_close > ha_open: score += 1.5
        if minus_di > plus_di: score -= 1
        if close < supertrend: score -= 1.5
        if ema5 < ema10 < ema20: score -= 1.5
        if macd < macd_signal: score -= 1
        if variance > variance_threshold and close < lower_bb: score -= 1.5
        if ema50 < ema200: score -= 1
        if smht < -1 and ha_close < ha_open: score -= 1.5

        # --- Signal Classification ---
        risk_buffer = close * 0.0075  # ~0.75% buffer
        vola_boost = variance * 0.1




        if score >= 4 :
            signal = f"Strong BUY ({round(score, 1)})"
            entry = close + close * 0.001  # buffer
            stop = lower_bb if lower_bb else (s1 if s1 else close - risk_buffer - vola_boost)
            target = upper_bb if upper_bb else (r1 if r1 else close + (close - stop) * 2)

        elif 2 <= score < 4:
            signal = f"BUY ({round(score, 1)})"
            entry = close + close * 0.0005
            stop = lower_bb if lower_bb else close - risk_buffer
            target = upper_bb if upper_bb else close + (close - stop) * 1.5

        elif score <= -4:
            signal = f"Strong SELL ({round(score, 1)})"
            entry = close # close - close * 0.001
            stop = upper_bb if upper_bb else (r1 if r1 else close + risk_buffer + vola_boost)
            if stop <= entry:
                risk = close * 0.01
                stop = entry + risk
            else:
                risk = stop - entry
            raw_target = entry - risk * 2
            target = lower_bb if lower_bb and lower_bb < entry else (
                s1 if s1 and s1 < entry else raw_target
            )

        elif -4 < score <= -2:
            signal = f"SELL ({round(score, 1)})"
            entry = close - close * 0.0005
            stop = upper_bb if upper_bb else (r1 if r1 else close + risk_buffer + vola_boost)
            if stop <= entry:
                risk = close * 0.01
                stop = entry + risk
            else:
                risk = stop - entry
            raw_target = entry - risk * 2
            target = lower_bb if lower_bb and lower_bb < entry else (
                s1 if s1 and s1 < entry else raw_target
            )



        # Reward/Risk
        rr = round((target - entry) / (entry - stop), 2) if entry and stop and target else ""



        signals.append(signal)
        scores.append(score)
        entries.append(round(entry, 2) if entry else "")
        targets.append(round(target, 2) if target else "")
        stops.append(round(stop, 2) if stop else "")
        rr_ratios.append(rr)

    return signals, scores, entries, targets, stops, rr_ratios

current_close = df_3min['Close'].iloc[-1]
current_high = df_3min['High'].iloc[-1]
current_low = df_3min['Low'].iloc[-1]

# 🔹 Get recent valid swing points (last N)
recent_swing_highs = df_3min['Swing_High'].dropna().tail(5)
recent_swing_lows = df_3min['Swing_Low'].dropna().tail(5)

def generate_smht_signal(df):
    signals = []
    for smht in df["SMHT"]:
        if smht > 2:
            signals.append("Strong Buy")
        elif smht < -2:
            signals.append("Strong Sell")
        else:
            signals.append("")
    return signals

# Usage:
df_3min["SMHT_Signal"] = generate_smht_signal(df_3min)

df_3min["HA_SW_High"] = df_3min["HA_SW_High"].apply(lambda x: "T" if x else "F")
df_3min["HA_SW_Low"]= df_3min["HA_SW_Low"].apply(lambda x: "T" if x else "F")
df_3min["Signal"], df_3min["Score"], df_3min["Entry"], df_3min["Target"], df_3min["StopLoss"], df_3min["RR"] = generate_signals_with_dynamic_ranges(df_3min)
#df_3min["EMA_20>50"] = np.where(df_3min["EMA_20"] > df_3min["EMA_50"], "T", "F")
df_3min["EMA_Status"] = (
    np.where(df_3min["EMA_5"] > df_3min["EMA_10"], "T", "F") +
    np.where(df_3min["EMA_10"] > df_3min["EMA_20"], "T", "F") +
    np.where(df_3min["EMA_20"] > df_3min["EMA_50"], "T", "F") +
    np.where(df_3min["EMA_50"] > df_3min["EMA_100"], "T", "F")
)
Entry_condition = (
    (df_3min["EMA_Status"] == "TTTT") &
    (df_3min["HA_Trend"] == "Uptrend") &
    --(df_3min["MACD"] > df_3min["MACD_Signal"]) &
    (df_3min["ADX"] > 20) &
    (df_3min["+DI"] > df_3min["-DI"])
)

Exit_condition = (
    (df_3min["EMA_Status"] == "FFFF") &
    (df_3min["HA_Trend"] == "Downtrend") &
    (df_3min["MACD"] < df_3min["MACD_Signal"]) &
    (df_3min["ADX"] > 20) &
    (df_3min["+DI"] < df_3min["-DI"])
)
df_3min["EE_Signal"] = np.where(Entry_condition, "Buy",
                       np.where(Exit_condition, "Sell", ""))


df_3min["Yest_H/L"]=""
# ✅ Add yesterday's high and low columns
df_3min['Prev_High'] = df_3min['High'].shift(1)
df_3min['Prev_Low'] = df_3min['Low'].shift(1)
df_3min['Prev_Close'] = df_3min['Close'].shift(1)

df_3min['Prev_High_Fill'] = df_3min['Swing_High'].ffill()
df_3min['Prev_Low_Fill'] = df_3min['Swing_Low'].ffill()

# df_3min['Break_High'] = (df_3min['Close'] > df_3min['Prev_High_Fill'])
# df_3min['Break_Low'] = (df_3min['Close'] < df_3min['Prev_Low_Fill'])

# Breakout conditions
df_3min['Break_High'] = (df_3min['Close'] > df_3min['Prev_High_Fill']) & (df_3min['Close'] > df_3min['Prev_Low_Fill'])
df_3min['Break_Low'] = (df_3min['Close'] < df_3min['Prev_Low_Fill']) & (df_3min['Close'] < df_3min['Prev_High_Fill'])
df_3min['Break_Prevs_SwingLow'] = (df_3min['Close'] > df_3min['Prev_Low_Fill']) & (df_3min['Close'] < df_3min['Prev_High_Fill'])
df_3min['Body'] = abs(df_3min['Close'] - df_3min['Open'])
df_3min['Strong_Break_High'] = df_3min['Break_High'] & (df_3min['Body'] > df_3min['ATR'] * 0.8)
atr_multiplier = 1.5
df_3min['ATR_Break_High'] = df_3min['Close'] > (df_3min['Prev_High_Fill'] + atr_multiplier * df_3min['ATR'])
df_3min['ATR_Break_Low'] = df_3min['Close'] < (df_3min['Prev_Low_Fill'] - atr_multiplier * df_3min['ATR'])
df_3min['Breakout_with_Momentum'] = (df_3min['Break_High']) & (df_3min['RSI'] > 60)
df_3min['Inside_Bar'] = (df_3min['High'] < df_3min['Prev_High']) & (df_3min['Low'] > df_3min['Prev_Low'])

conditions =[
    df_3min['Break_High'],
    df_3min['Break_Low'],
    df_3min['Break_Prevs_SwingLow'],
    df_3min['Strong_Break_High'],
    df_3min['ATR_Break_High'],
    df_3min['ATR_Break_Low'],
    df_3min['Breakout_with_Momentum']
    ]


choices = ['SwingHigh-Breakout-BUY', 'SWingLow-Breakdown-SELL','Swinglow_Breakout-ShortBuy','Strong_Break_High-BUY','ATR_Break_High-BUY','ATR_Break_Low-SELL','Breakout_with_Momentum-StrongBUY']

df_3min['Swing_Signal'] = np.select(conditions, choices, default='Hold')

import pandas as pd

def generate_signals_with_wick_analysis(df, body_pct_threshold=0.6, volume_filter=False):
    """
    Generate buy/sell signals based on body size and wick analysis.
    """
    df['Body'] = abs(df['Close'] - df['Open'])
    df['Range'] = df['High'] - df['Low']
    df['Upper_Wick'] = df['High'] - df[['Open', 'Close']].max(axis=1)
    df['Lower_Wick'] = df[['Open', 'Close']].min(axis=1) - df['Low']
    df['Body_pct'] = df['Body'] / df['Range']
    df['Avg_Body_pct'] = df['Body_pct'].rolling(20).mean()
    if volume_filter:
        df['Avg_Vol'] = df['Volume'].rolling(20).mean()

    def signal(row):
        if row['Range'] == 0 or pd.isna(row['Avg_Body_pct']):
            return None
        if volume_filter and row['Volume'] < row['Avg_Vol']:
            return None

        # BUY signal
        if (row['Close'] > row['Open'] and
            row['Body_pct'] > body_pct_threshold and
            row['Lower_Wick'] > row['Upper_Wick']):
            return 'Buy'

        # SELL signal
        elif (row['Open'] > row['Close'] and
              row['Body_pct'] > body_pct_threshold and
              row['Upper_Wick'] > row['Lower_Wick']):
            return 'Sell'

        return None

    df['Body_Signal'] = df.apply(signal, axis=1)
    return df

df_3min=generate_signals_with_wick_analysis(df_3min, body_pct_threshold=0.6, volume_filter=False)
df_3min["Body_Signal"] = df_3min["Body_Signal"].fillna("Hold")
df_3min["Swing_Signal"] = df_3min["Swing_Signal"] + "_BODY_ "+df_3min["Body_Signal"] +"JP_"+ df_3min["JP_Signal"]


def swing_trading_signals(df):
    # Calculate required columns
    df['candle_size'] = df['Close'] - df['Open']
    df['avg_candle_size'] = df['candle_size'].rolling(window=20).mean()
    df['avg_volume_20'] = df['Volume'].rolling(window=20).mean()

    # Condition 1: Volume spike
    df['vol_spike'] = df['Volume'] > 1.5 * df['avg_volume_20']


    # Condition 2: Strong bullish candle
    df['bullish_body'] = df['Close'] > df['Open']
    df['strong_bullish_candle'] = df['candle_size'] > 1.2 * df['avg_candle_size']

    # Condition 3: Breakout above previous high
    df['breakout'] = df['Close'] > df['High'].shift(1)

    if nifty == "^NSEI":
        df['buy_signal'] = df['bullish_body'] & df['strong_bullish_candle'] & df['breakout']
    else:
        df['buy_signal'] = df['vol_spike'] & df['bullish_body'] & df['strong_bullish_candle'] & df['breakout']

    # Swing low (for stop loss)
    df['swing_low'] = df['Low'].rolling(window=3, center=True).min()

    # Entry, Stop Loss, Target
    df['entry_price'] = df['Close'].where(df['buy_signal'])
    df['stop_loss'] = df['swing_low'].where(df['buy_signal'])
    df['risk'] = df['entry_price'] - df['stop_loss']
    df['target'] = df['entry_price'] + 1.5 * df['risk']

    return df

#"entry_price","target","risk","stop_loss"
#Important Entry and stop lost
df_3min = swing_trading_signals(df_3min)
# Combine all into a single string column
df_3min['trade_setup'] = df_3min.apply(
    lambda row: f"Entry: {row['entry_price']:.2f}, SL: {row['stop_loss']:.2f}, Risk: {row['risk']:.2f}, Target: {row['target']:.2f}"
    if row['buy_signal'] else "",
    axis=1
)
#buy_signals = df_3min[df_3min['buy_signal'] == True]

# ✅ Check for breakouts based on yesterday's high and low
df_3min['Yesterday_High_Breaks_Close'] = df_3min['Close'] >df_3min['Prev_High']
df_3min['Yesterday_Low_Breaks_Close'] = df_3min['Close'] < df_3min['Prev_Low']
df_3min['Yesterday_Low_Breaks_Low'] = df_3min['Low'] < df_3min['Prev_Low']
df_3min['Yesterday_High_Beaks_PrevHigh'] = df_3min['High'] > df_3min['Prev_High']
df_3min['Yesterday_Low_Beaks_PrevHigh'] = df_3min['Low'] > df_3min['Prev_High']
df_3min['Today_High_Breaks_SwingHigh'] = df_3min['Close'] > df_3min['Prev_High_Fill']
df_3min['Today_High_Breaks_SwingLow'] = df_3min['Low'] < df_3min['Prev_Low_Fill']
df_3min['Today_SwinghLow_Breakout'] = df_3min['Close'] > df_3min['Prev_High_Fill']
df_3min['Today_CloseBreaks_BB_Upper'] = df_3min['Close'] > df_3min['Upper_BB']
df_3min['Today_LowBreakdown_BB_Lower'] = df_3min['Low'] < df_3min['Lower_BB']

df_3min['Yest_H/L'] = np.where(df_3min['Yesterday_High_Breaks_Close'], 'Current Close breaks Previous High- BUY',
                             np.where(df_3min['Yesterday_Low_Breaks_Close'], 'Current Close lessthan Previous Low -Sell',
                              np.where(df_3min['Today_High_Breaks_SwingHigh'], 'Current Close breaks Swingh High -BUY',
                               np.where(df_3min['Today_High_Breaks_SwingLow'], 'Current Close breakdown Swing low -Sell',
                                  np.where(df_3min['Yesterday_High_Beaks_PrevHigh'], 'Current High Breakout PrevHigh -BUY',
                                       np.where(df_3min['Today_SwinghLow_Breakout'], 'Current Close Breaks SwinghLow -BUY',

                                                  np.where(df_3min['Today_CloseBreaks_BB_Upper'], 'Current Close Breakout BB_UPPER -BUY',
                                                             np.where(df_3min['Today_LowBreakdown_BB_Lower'], 'Current Low Breakdown BB_Lower -SELL',
                             np.where(df_3min['Yesterday_Low_Breaks_Low'], 'Current Low lessthan Previous Low -Sell', 'No Break')))))))))


df_3min["strong_buy"] = (
    (df_3min["EMA_5"] > df_3min["EMA_10"]) &
    (df_3min["EMA_10"] > df_3min["EMA_20"]) &
    (df_3min["ADX"] > 25) &
    (df_3min["+DI"] > df_3min["-DI"]) &
    (df_3min["Supertrend"] < df_3min["Close"]) &
    #(df_3min["RSI"] < 60) &
    (df_3min["HA_SW_Low"] == "T") &
    (df_3min["Close"] > df_3min["S1"])  # Confirmed above support

    )
df_3min["strong_sell"] = (
    (df_3min["EMA_5"] < df_3min["EMA_10"]) &
    (df_3min["EMA_10"] < df_3min["EMA_20"]) &
    (df_3min["ADX"] > 25) &
    (df_3min["-DI"] > df_3min["+DI"]) &
    (df_3min["Supertrend"] > df_3min["Close"]) &
    #(df_3min["RSI"] > 40) &
    (df_3min["HA_SW_High"] == "T") &
    (df_3min["Close"] < df_3min["R1"])  # Rejected at resistance
)
df_3min["Final_Signal"] = np.where(
    df_3min["strong_buy"], "STRONG BUY",
    np.where(df_3min["strong_sell"], "STRONG SELL", df_3min["Yest_H/L"])
    )
# Use previous row's S1 and R1
df_3min["S1_prev"] = df_3min["S1"].shift(1)
df_3min["R1_prev"] = df_3min["R1"].shift(1)
df_3min["ATR"] = atr(df_3min, 10)


# Step 1: Calculate Swing High/Low (exclude current candle)
df_3min["Swing_High_5"] = df_3min["High"].shift(1).rolling(window=5).max()
df_3min["Swing_Low_5"] = df_3min["Low"].shift(1).rolling(window=5).min()
# Signal for candel pattern

#df_today=df_3min[["Close","High","Low", "+DI", "-DI","DI_Diff","Variance", "S1", "R1", "S2","R2","HA_Signal", "ADX", "ADX_Incr", "Signal", "EMA_5", "EMA_10", "EMA_20","Golden/Death"]]

df_3min=df_3min[["Open","Close","High","Low", "Signal","S1","R1","S2","R2","Swing_Signal","Final_Signal","Swing_High","Swing_Low","ADX_Incr","ADX","+DI", "-DI","HA_SW_High","HA_SW_Low","HA_Trend","trade_setup"]]
#df_3min=df_3min[["Open","Close","High","Low", "Signal","S1","R1","S2","R2","Swing_Signal","Final_Signal","Swing_High","Swing_Low","ADX_Incr","ADX","+DI", "-DI","trade_setup"]]
# filename = f"nifty50BB_{nifty}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
# df_3min.to_csv(filename)
#df_3min = df_3min[(df_3min["EE_Signal"] == "Buy") & (df_3min["HA_SW_Low"] == "T")] # for buy
#df_3min = df_3min[(df_3min["EE_Signal"] == "Buy")] # for buy
#df_3min = df_3min[(df_3min["HA_SW_Low"] == "T")] # for buy
#df_3min = df_3min[(df_3min["HA_SW_High"] == "T")] # for sell
# Find last non-empty trade_setup
last_trade = df_3min[df_3min['trade_setup'] != '']['trade_setup'].iloc[-1]
print(f"Last Trade Setup: {last_trade}")

df_3min.tail(10)



[*********************100%***********************]  1 of 1 completed


Last Trade Setup: Entry: 25033.00, SL: 25020.00, Risk: 13.00, Target: 25052.50


,Open,Close,High,Low,Signal,S1,R1,S2,R2,Swing_Signal,...,Swing_High,Swing_Low,ADX_Incr,ADX,+DI,-DI,HA_SW_High,HA_SW_Low,HA_Trend,trade_setup
Datetime,,,,,,,,,,,,,,,,,,,,,
2025-05-16 15:10:00+05:30,25033.0,25033.0,25034.0,25030.0,Strong BUY (4.5),25030.0,25034.0,25028.0,25036.0,Swinglow_Breakout-ShortBuy_BODY_ HoldJP_NEU,...,25034.0,NaN,True,13.0,19.42,12.68,F,F,Sideways,
2025-05-16 15:12:00+05:30,25034.0,25030.0,25034.0,25029.0,BUY (3.0),25028.0,25033.0,25026.0,25036.0,Swinglow_Breakout-ShortBuy_BODY_ HoldJP_NEU,...,25034.0,NaN,True,13.0,18.61,12.98,F,F,Sideways,
2025-05-16 15:14:00+05:30,25030.0,25021.0,25030.0,25021.0,No Signal,25018.0,25027.0,25015.0,25033.0,Swinglow_Breakout-ShortBuy_BODY_ HoldJP_NEU,...,NaN,NaN,True,12.0,17.23,18.63,F,F,Sideways,
2025-05-16 15:16:00+05:30,25023.0,25018.0,25023.0,25017.0,No Signal,25015.0,25021.0,25013.0,25025.0,SWingLow-Breakdown-SELL_BODY_ HoldJP_NEU,...,NaN,NaN,False,12.0,16.35,21.07,F,F,Downtrend,
2025-05-16 15:18:00+05:30,25018.0,25017.0,25019.0,25015.0,SELL (-2.5),25015.0,25019.0,25013.0,25021.0,SWingLow-Breakdown-SELL_BODY_ HoldJP_NEU,...,NaN,NaN,True,13.0,15.78,22.09,F,F,Downtrend,
2025-05-16 15:20:00+05:30,25018.0,25015.0,25018.0,25011.0,Strong SELL (-4.0),25012.0,25019.0,25008.0,25022.0,Swinglow_Breakout-ShortBuy_BODY_ HoldJP_NEU,...,NaN,25011.0,True,14.0,14.80,24.26,F,F,Downtrend,
2025-05-16 15:22:00+05:30,25015.0,25012.0,25018.0,25011.0,Strong SELL (-4.0),25010.0,25017.0,25007.0,25021.0,Swinglow_Breakout-ShortBuy_BODY_ HoldJP_NEU,...,NaN,25011.0,True,14.0,13.87,22.74,F,F,Downtrend,
2025-05-16 15:24:00+05:30,25012.0,25016.0,25019.0,25011.0,SELL (-2.5),25011.0,25019.0,25007.0,25023.0,Swinglow_Breakout-ShortBuy_BODY_ HoldJP_NEU,...,25019.0,NaN,True,15.0,13.77,21.11,F,T,Sideways,
2025-05-16 15:26:00+05:30,25016.0,25013.0,25018.0,25012.0,Strong SELL (-4.0),25010.0,25016.0,25008.0,25020.0,Swinglow_Breakout-ShortBuy_BODY_ HoldJP_NEU,...,NaN,NaN,True,15.0,13.02,19.96,F,T,Sideways,


In [5]:
# @title MulipleStock_Signals
import yfinance as yf
import pandas as pd
import ta

# Load stock symbols
df = pd.read_csv('/content/ALL_IND_Stocks.csv')
stock_symbols = df['Symbol'].tolist()

breakout_stocks = []

for symbol in stock_symbols:
    try:
        stock = yf.Ticker(symbol)
        df = stock.history(period="6mo")

        if df.empty:
            continue

        # Moving Averages
        df["SMA_7"] = df["Close"].rolling(7).mean()
        df["SMA_10"] = df["Close"].rolling(10).mean()
        df["SMA_20"] = df["Close"].rolling(20).mean()
        df["SMA_50"] = df["Close"].rolling(50).mean()

        df["EMA_7"] = df["Close"].ewm(span=7, adjust=False).mean()
        df["EMA_10"] = df["Close"].ewm(span=10, adjust=False).mean()
        df["EMA_20"] = df["Close"].ewm(span=20, adjust=False).mean()
        df["EMA_50"] = df["Close"].ewm(span=50, adjust=False).mean()

        df["Avg_Volume_7"] = df["Volume"].rolling(7).mean()
        df["Avg_Volume_20"] = df["Volume"].rolling(20).mean()
        df["Avg_Volume_30"] = df["Volume"].rolling(30).mean()
        df["Avg_Volume_90"] = df["Volume"].rolling(90).mean()
        df['Vol_spike'] = df['Volume'] > 1.5 * df['Avg_Volume_7']

        # ADX
        df['ADX'] = ta.trend.ADXIndicator(df['High'], df['Low'], df['Close'], window=14).adx()

        # Bollinger Bands
        bb = ta.volatility.BollingerBands(df['Close'], window=20, window_dev=2)
        df['BB_High'] = bb.bollinger_hband()
        df['BB_Low'] = bb.bollinger_lband()
        df['BB_Mid'] = bb.bollinger_mavg()

        # Candlestick metrics
        df['Body_Size'] = abs(df['Close'] - df['Open'])
        df['Avg_Body_Size'] = df['Body_Size'].rolling(20).mean()
        df['High_Low_Range'] = df['High'] - df['Low']

        latest = df.iloc[-1]
        previous = df.iloc[-2]

        score = 0
        signal = "HOLD"

        # Price action
        if latest['Close'] > previous['Close']:
            score += 1
        elif latest['Close'] < previous['Close']:
            score -= 1

        # Volume breakout
        for vol_avg, pts in zip(['Avg_Volume_7', 'Avg_Volume_20', 'Avg_Volume_30', 'Avg_Volume_90'], [1, 2, 3, 4]):
            if latest['Volume'] > latest[vol_avg] * 1.5:
                score += pts

        if latest['Volume'] < latest['Avg_Volume_30'] * 0.7:
            score -= 2

        # Moving average alignment
        if latest['Close'] > latest['EMA_7'] > latest['EMA_10'] > latest['EMA_20'] > latest['EMA_50']:
            score += 3
        elif latest['Close'] > latest['EMA_7'] > latest['EMA_10']:
            score += 2

        if latest['Close'] > latest['SMA_7'] > latest['SMA_10'] > latest['SMA_20'] > latest['SMA_50']:
            score += 3
        elif latest['Close'] > latest['SMA_7'] > latest['SMA_10']:
            score += 2

        # ADX strength
        if latest['ADX'] > 25:
            score += 2
        elif latest['ADX'] < 20:
            score -= 1

        # Bollinger band support
        if latest['Close'] > latest['BB_Low'] * 1.02:
            score += 2
        elif latest['Close'] < latest['BB_Low']:
            score -= 2

        # Strong body + volume = confirmation
        if latest['Body_Size'] > 1.5 * latest['Avg_Body_Size'] and latest['Volume'] > latest['Avg_Volume_20']:
            score += 2

        # Final Signal
        if score >= 10:
            signal = "STRONG BUY"
        elif 6 <= score < 10:
            signal = "BUY"
        elif score <= -5:
            signal = "STRONG SELL"
        elif -5 < score <= -2:
            signal = "SELL"

        breakout_stocks.append((
            symbol, latest['Close'],previous['Close'], latest['Volume'],latest['Vol_spike'],
            latest['EMA_7'], latest['EMA_10'], latest['EMA_20'], latest['EMA_50'],
            latest['SMA_7'], latest['SMA_10'], latest['SMA_20'], latest['SMA_50'],
            latest['ADX'], latest['BB_Low'], latest['BB_High'],
            latest['Body_Size'], latest['Avg_Body_Size'], latest['High_Low_Range'],
            score, signal
        ))

    except Exception as e:
        print(f"Error for {symbol}: {e}")

# Final dataframe
result_df = pd.DataFrame(breakout_stocks, columns=[
    "Symbol", "Close","Prev_Close", "Volume",'Vol_spike',
    "EMA_7", "EMA_10", "EMA_20", "EMA_50",
    "SMA_7", "SMA_10", "SMA_20", "SMA_50",
    "ADX", "BB_Low", "BB_High",
    "Body_Size", "Avg_Body_Size", "High_Low_Range",
    "Score", "Signal"
])

result_df = result_df.sort_values(by=["Score", "Volume"], ascending=[False, False])
result_df.to_csv("/content/enhanced_breakout_signals.csv", index=False)
print("File saved at /content/enhanced_breakout_signals.csv")
print(result_df.head(10))


Error for ISEC.NS: negative dimensions are not allowed


Exception ignored from cffi callback <function buffer_callback at 0x7a875ec0b6a0>:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/curl_cffi/curl.py", line 63, in buffer_callback
    @ffi.def_extern()
    
KeyboardInterrupt: 


File saved at /content/enhanced_breakout_signals.csv
            Symbol         Close    Prev_Close    Volume  Vol_spike  \
117  COCHINSHIP.NS   2035.599976   1812.099976  33598999       True   
317     MAZDOCK.NS   3522.399902   3182.399902  16561103       True   
169        GRSE.NS   2481.699951   2258.800049  14393037       True   
156       NYKAA.NS    201.800003    197.320007  11944254      False   
208         HAL.NS   5127.700195   4864.799805   6753397      False   
37     APARINDS.NS   8228.000000   7941.000000    331664      False   
72          BEL.NS    363.899994    350.399994  68855818       True   
357     PAGEIND.NS  47830.000000  46940.000000     45705       True   
227       IRCON.NS    188.880005    177.580002  28027854       True   
327        NBCC.NS    111.080002    106.720001  27415877      False   

            EMA_7        EMA_10        EMA_20        EMA_50         SMA_7  \
117   1727.563076   1674.058864   1585.807787   1491.585381   1653.271432   
317   3145.